<a href="https://colab.research.google.com/github/himasri-24/Diabetic-Vision-Impairment/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile

# Define constants
IMG_SIZE = (128, 128)  # Image size
BATCH_SIZE = 16  # Batch size
EPOCHS = 5  # Number of epochs
ZIP_FILE_PATH = "/content/archive.zip"  # Corrected file path
EXTRACTED_FOLDER = "/mnt/data/extracted_images"  # Folder where images will be extracted

# Ensure the ZIP file exists before proceeding
if not os.path.exists(ZIP_FILE_PATH):
    raise FileNotFoundError(f"ZIP file '{ZIP_FILE_PATH}' not found. Please upload the correct file.")

try:
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACTED_FOLDER)
except zipfile.BadZipFile:
    raise zipfile.BadZipFile(f"Error: '{ZIP_FILE_PATH}' is not a valid ZIP file. Please check the file format.")

# Update dataset path
DATASET_PATH = EXTRACTED_FOLDER

# Ensure the dataset directory exists
if not os.path.exists(DATASET_PATH) or not os.path.isdir(DATASET_PATH):
    raise FileNotFoundError(f"Dataset path '{DATASET_PATH}' not found. Please check the extraction process.")

# Debugging: Print dataset directory structure
print("Dataset directory structure:", os.listdir(DATASET_PATH))

# Set up data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Automatically determine steps_per_epoch & validation_steps
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")

# Build a custom CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Dynamically set output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Add Early Stopping to prevent unnecessary training
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(train_generator,
          validation_data=val_generator,
          epochs=EPOCHS,
          steps_per_epoch=steps_per_epoch,
          validation_steps=validation_steps,
          callbacks=[early_stopping])

# Save the model
model.save('diabetic_retinopathy_cnn.h5')

print("Model training complete and saved as 'diabetic_retinopathy_cnn.h5'")


Dataset directory structure: ['Proliferate DR', 'Severe DR', 'Mild DR', 'Healthy', 'Moderate DR']
Found 2200 images belonging to 5 classes.
Found 550 images belonging to 5 classes.
Steps per epoch: 138, Validation steps: 35


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 81s 573ms/step - accuracy: 0.4762 - loss: 1.3413 - val_accuracy: 0.6382 - val_loss: 0.9513
Epoch 2/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 75s 519ms/step - accuracy: 0.6228 - loss: 1.0296 - val_accuracy: 0.6782 - val_loss: 0.9148
Epoch 3/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 71s 512ms/step - accuracy: 0.6503 - loss: 0.9410 - val_accuracy: 0.6745 - val_loss: 0.8675
Epoch 4/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 73s 526ms/step - accuracy: 0.6566 - loss: 0.9406 - val_accuracy: 0.6855 - val_loss: 0.8571
Epoch 5/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 69s 500ms/step - accuracy: 0.6774 - loss: 0.8897 - val_accuracy: 0.7036 - val_loss: 0.8428


Model training complete and saved as 'diabetic_retinopathy_cnn.h5'


You can download the saved model file `diabetic_retinopathy_cnn.h5` using the following code cell: